### Generators:
- a function that uses the **`yield`** statement, is called a **`generator function`**.
- when we call the function it returns a **`generator`** object.
- We can think of functions that contain the yield statement as **`generator factories`**.
- Generators **`are`** iterators, and can be used in the same way (for loops, comprehensions).
- Generators become exhausted once the function returns a value.

Suppose we need to read in a logfile and print out the lines labeled with "warning"

In [5]:
def matching_lines_from_file(path, pattern):
    with open(path) as handle:
        for line in handle:
            if pattern in line:
                yield line.rstrip('\n')

for line in matching_lines_from_file('logfile.log', 'WARNING'):
    print(line)

Suppose we need the information in a dictionary

In [6]:
def parse_log_records(lines):
    for line in lines:
        level, message = line.split(': ', 1)
        yield {'level': level, 'message': message}

log_lines = matching_lines_from_file('logfile.log', 'WARNING')
for record in parse_log_records(log_lines):
    print(record)

{'level': 'WARNING', 'message': 'Disk Usage exceeding 85%'}
{'level': 'WARNING', 'message': "Almost out of Lion's Mane mushroom"}


Assume we have a text file containing data on residential house sales. Address, square_feet and price is recorded for each sale, each on a separate line.

In [8]:
def lines_from_file(path):
    with open(path) as handle:
        for line in handle:
                yield line.rstrip('\n')

def house_records(lines):
    record ={}
    for line in lines:
        if line == '':
            yield record
            record = {}
            continue
        key, value = line.split(': ', 1)
        record[key] = value
    yield record


def house_records_from_file(path):
    lines = lines_from_file(path)
    for house in house_records(lines):
        yield house

for house in house_records_from_file('house_sale_data.txt'):
    print(house)


{'address': '1423 99th Ave', 'square_feet': '1705', 'price_usd': '340210'}
{'address': '24257 Pueblo Dr', 'square_feet': '2305', 'price_usd': '170210'}
{'address': '127 Cochran', 'square_feet': '2068', 'price_usd': '320500'}


### `TIP`
When one generator function internally calls another we can use `yield from` such as below

In [10]:
# Instead of this
def house_records_from_file(path):
    lines = lines_from_file(path)
    for house in house_records(lines):
        yield house

# we write this
def house_records_from_file(path):
    lines = lines_from_file(path)
    # we can use yield from because house_records 
    yield from house_records(lines)

for house in house_records_from_file('house_sale_data.txt'):
    print(house)
        

{'address': '1423 99th Ave', 'square_feet': '1705', 'price_usd': '340210'}
{'address': '24257 Pueblo Dr', 'square_feet': '2305', 'price_usd': '170210'}
{'address': '127 Cochran', 'square_feet': '2068', 'price_usd': '320500'}


In [2]:
import math

# Creating an iterator
class FactIter:
    
    def __init__(self, n):
        self.n = n
        self.i = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        else:
            result = math.factorial(self.i)
            self.i += 1
            return result

fact_iter = FactIter(5)
list(fact_iter)

[1, 1, 2, 6, 24]

In [ ]:
def fact():
    i = 0
    def inner():
        nonlocal i
        result = math.factorial(i)
        i += 1
        return result
    return inner

fact_iter = iter(fact(), math.factorial(5))

We can replace the iterator and closure above with a generator

In [6]:
# Creating a generator
def factorials(n):
    for i in range(n):
        yield math.factorial(i)

fact_iter = factorials(5)

list(fact_iter)

[1, 1, 2, 6, 24]

### Fibonacci sequence

In [7]:
# iterator

def fib(n):
    fib_0 = 1
    fib_1 = 1
    for i in range(n-1):
        fib_0, fib_1 = fib_1, fib_0 + fib_1
    return fib_1

class FibIter:
    def __init__(self, n):
        self.n = n
        self.i = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        else:
            result = fib(self.i)
            self.i += 1
            return result

In [ ]:
# generator
def fib(n):
    fib_0 = 1
    yield fib_0
    fib_1 = 1
    yield fib_1
    for i in range(n-2):
        fib_0, fib_1 = fib_1, fib_0 + fib_1
        yield fib_1

### Making an iterable from a generator

In [11]:
def squares(n):
    for i in range(n):
        yield i ** 2


class Squares:

    def __init__(self, n):
        self.n = n

    def __iter__(self):
        return Squares.squares(self.n)
    
    @staticmethod
    def squares(n):
        for i in range(n):
            yield i ** 2

# now we can iterate through an instance
# of Squares without exhausting it. 

sq = Squares(5)
l1 = [num for num in sq]
l2 = [num for num in sq]
print(l1, l2)

[0, 1, 4, 9, 16] [0, 1, 4, 9, 16]


## Card Deck Generator

In [1]:
from collections import namedtuple

Card = namedtuple('Card', 'rank suit')
SUITS = ('Spades', 'Hearts', 'Diamonds', 'Clubs')
RANKS = tuple(range(2, 11)) + tuple('JQKA')

def card_gen():
    for i in range(len(SUITS) * len(RANKS)):
        suit = SUITS[i] // len(RANKS)
        rank = RANKS[i] % len(RANKS)
        card = Card(rank, suit)
        yield card

# The function above can be written in an easier way
def card_gen():
    for suit in SUITS:
        for rank in RANKS:
            yield Card(rank, suit)


# The function above is a generator, but not yet an iterable,
Card = namedtuple('Card', 'rank suit')

class CardDeck:
    SUITS = ('Spades', 'Hearts', 'Diamonds', 'Clubs')
    RANKS = tuple(range(2, 11)) + tuple('JQKA')


    def __iter__(self):
        return CardDeck.card_gen()
    
    def __reverse__(self):
       return CardDeck.reversed_card_gen() 

    @staticmethod
    def card_gen():
        for suit in CardDeck.SUITS:
            for rank in CardDeck.RANKS:
                yield Card(rank, suit)
    
    @staticmethod
    def reversed_card_gen():
        for suit in reversed(CardDeck.SUITS):
            for rank in reversed(CardDeck.RANKS):
                yield Card(rank, suit)


## Generator expressions
- Similar syntax as list comprehensions, but is wrapped inside **`()`** instead of `[]`
- Returns a Generator
- Lazy evaluation - object creation is delayed until requested by **`next()`**
- Has local scope

In [1]:
mygenerator = (i for i in range(10) if i % 2 == 0)
for i in mygenerator:
    print(i)

0
2
4
6
8


## Yield from

In [ ]:
# EXAMPLE 1
def matrix(n):
    gen = ((i * j for j in range(1, n+1)) for i in range(1, n+1))
    return gen

def matrix_iterator(n):
    for row in matrix(n):
        for item in row:
            yield item

# we can change the above to a more simple expression
def matrix_iterator(n):
    for row in matrix(n):
        yield from row

# EXAMPLE 2
file_1 = 'car-brands-1.txt'
file_2 = 'car-brands-2.txt'
file_3 = 'car-brands-3.txt'
files = file_1 + file_2 + file_3


# we split the cleaning and looping over the files
# into two separate functions - separations of concerns
def gen_clean_data(file):
    with open(file) as f:
        for row in f:
            yield row.strip('\n')

def brands(*files):
    for f_name in files:
        yield from gen_clean_data(f_name)
